In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Clean entity

In [9]:
def clean_entities(db_path: str) -> None:
    """
    Dọn bảng Entities trong CSDL SQLite:
    1. Xoá entity_text < 2 ký tự
    2. Xoá thực thể không phải 'STOCK' (confidence <> 'high'
       & không có ≥3 IN HOA liên tiếp & chỉ 1 từ)
    """
    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()

        # ------ Thống kê trước khi xoá ------
        total_before = cur.execute("SELECT COUNT(*) FROM Entities").fetchone()[0]
        print(f"Tổng số hàng ban đầu: {total_before:,}")

        # 1) Xoá entity_text ngắn < 2 ký tự
        cur.execute("""
            DELETE FROM Entities
            WHERE length(trim(entity_text)) < 2;
        """)
        print(f"Đã xoá (entity_text < 2 ký tự): {conn.total_changes:,}")

        # 2) Xoá thực thể không phải STOCK
        cur.execute("""
            DELETE FROM Entities
            WHERE confidence <> 'high'                      -- confidence khác 'high'
              AND entity_text NOT GLOB '*[A-Z][A-Z][A-Z]*' -- KHÔNG có ≥3 in hoa liên tiếp
              AND entity_text NOT LIKE '% %';               -- KHÔNG có khoảng trắng (chỉ 1 từ)
        """)
        print(f"Đã xoá (không phải STOCK): {conn.total_changes - 1:,}")  # trừ lần xoá trước

        # ------ Thống kê sau khi xoá ------
        total_after = cur.execute("SELECT COUNT(*) FROM Entities").fetchone()[0]
        print(f"Tổng số hàng còn lại: {total_after:,}")

In [10]:
path_db = '/content/drive/MyDrive/statlearning/KB/stock_insights.db'
clean_entities(path_db)

Tổng số hàng ban đầu: 369,436
Đã xoá (entity_text < 2 ký tự): 9
Đã xoá (không phải STOCK): 221,173
Tổng số hàng còn lại: 148,262


# Khảo sát các entity

In [11]:
import sqlite3, pandas as pd

conn = sqlite3.connect('/content/drive/MyDrive/statlearning/KB/stock_insights.db')
query = """
SELECT entity_type, COUNT(*) AS total
FROM   Entities
WHERE  entity_type IN ('EVENT','RISK','PRICE_ACTION')
GROUP  BY entity_type;
"""
df = pd.read_sql_query(query, conn)
print(df)
conn.close()


    entity_type  total
0         EVENT   1628
1  PRICE_ACTION   5837
2          RISK    618


In [12]:
conn = sqlite3.connect(path_db)
types = ['EVENT', 'RISK', 'PRICE_ACTION']
lists = {}
for t in types:
    df = pd.read_sql_query(
        "SELECT DISTINCT entity_text FROM Entities WHERE entity_type=? ORDER BY entity_text",
        conn, params=[t]
    )
    lists[t] = df['entity_text'].tolist()

In [16]:
from pathlib import Path

out_dir = Path("entity_lists")      # thư mục chứa file xuất
out_dir.mkdir(exist_ok=True)

for etype, items in lists.items():
    file_path = out_dir / f"{etype.lower()}_list.txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n".join(items))   # mỗi entity một dòng
    print(f"Đã tạo {file_path} ({len(items):,} dòng)")


Đã tạo entity_lists/event_list.txt (1,229 dòng)
Đã tạo entity_lists/risk_list.txt (442 dòng)
Đã tạo entity_lists/price_action_list.txt (3,292 dòng)


# Tính Sentiment Score và gán nhãn

In [42]:
path_signal = 'signal_dict_rescored.json'

In [43]:
import json
with open(path_signal, "r", encoding="utf-8") as f:
    signal_dict: dict[str, float] = json.load(f)

In [44]:
len(signal_dict)

1217

In [ ]:
RISK_EXTRA   = ["bất thành","không khớp","cảnh báo","thu hồi","hủy","điều tra","trì hoãn"]
for word in RISK_EXTRA:
    signal_dict[word] = -1.0

In [46]:
def score_sentence_v2(text):
    txt = text.lower()
    score = 0.0
    for key, w in signal_dict.items():
        if key in txt:
            # (a) phủ định trực tiếp
            if re.search(r"\bkhông\s+" + re.escape(key), txt) or \
               re.search(r"\bbất\s+" + re.escape(key.split()[0]), txt):
                w = -abs(w)             # đảo dấu
            score += w
    # (b) cap điểm tuyệt đối
    if score > 4:  score = 4
    if score < -4: score = -4
    return score

In [51]:
def label_from_score(score: float, pos_th=+0.2, neg_th=-0.2) -> str:
    if score > pos_th:
        return "positive"
    elif score < neg_th:
        return "negative"
    else:
        return "neutral"

In [52]:
conn = sqlite3.connect(path_db)
c = conn.cursor()
c.execute("""
    SELECT DISTINCT s.sentence_id, s.sentence_text
    FROM   Sentences  AS s
    JOIN   Entities   AS e USING(sentence_id)
    WHERE  e.entity_type IN ('STOCK', 'COMPANY')
      AND  e.confidence  = 'high'
    ORDER  BY s.sentence_id
    LIMIT  20;
""")
rows = c.fetchall()
for sid, text in rows:
    score = score_sentence_v2(text)
    label = label_from_score(score)
    print(text, score, label)

    # ---- (tuỳ chọn) cập nhật bảng ----
    # cur.execute("""
    #     UPDATE Sentences
    #     SET sentiment_score = ?, sentiment_label = ?
    #     WHERE sentence_id   = ?;
    # """, (score, label, sid))

# conn.commit()   # ← bỏ comment nếu thực sự muốn ghi DB
conn.close()

Con gái bầu Đức gom bất thành 4 triệu cp HAG vì không xoay được tiền. -2.0 negative
Bà Đoàn Hoàng Anh, con gái ông Đoàn Nguyên Đức, không mua được bất kỳ cổ phiếu nào trong số 4 triệu cổ phiếu HAG đăng ký, do không thu xếp được tài chính cá nhân. -2.8 negative
Theo công bố thông tin, bà Đoàn Hoàng Anh- con gái ông Đoàn Nguyên Đức(bầu Đức), Chủ tịch HĐQT CTCP Hoàng Anh Gia Lai (HAGL, HOSE:HAG)- không mua được cổ phiếu nào trong số 4 triệu cpHAG đăng ký từ ngày 14/04-13/05. -1.8 negative
Sau giao dịch bất thành, bà Hoàng Anh vẫn giữ nguyên tỷ lệ sở hữu 1.32%, tương ứng 14 triệu cp HAG. -1.0 negative
Trước đó, từ 30/12/2024-21/01/2025, bà đã mua thành công 1 triệu cpHAG qua khớp lệnh để nâng sở hữu như hiện tại. 2.8000000000000003 positive
Hiện tại, ông Đoàn Nguyên Đức là cổ đông lớn nhất của HAGL với tỷ lệ nắm giữ 30.26%, tương đương gần 320 triệu cp. 0.0 neutral
Tính chung cả gia đình bầu Đức đang nắm giữ gần 334 triệu cp HAG, chiếm khoảng 31.58% vốn điều lệ Công ty. 0.0 neutral
Trên th

In [53]:
def update_sentiments(db_path: str) -> None:
    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()

        query = """
        SELECT DISTINCT s.sentence_id, s.sentence_text
        FROM   Sentences AS s
        JOIN   Entities  AS e ON e.sentence_id = s.sentence_id
        WHERE  e.entity_type IN ('STOCK','COMPANY')
          AND  e.confidence  = 'high';
        """
        rows = cur.execute(query).fetchall()
        print(f"Đang cập nhật {len(rows):,} câu…")

        for sid, text in rows:
            sc  = score_sentence_v2(text)
            lab = label_from_score(sc)
            cur.execute("""UPDATE Sentences
                           SET sentiment_score = ?,
                               sentiment_label = ?
                           WHERE sentence_id   = ?""",
                        (sc, lab, sid))

        conn.commit()
        print("Hoàn tất cập nhật.")

In [54]:
update_sentiments(path_db)

Đang cập nhật 28,599 câu…
Hoàn tất cập nhật.


In [55]:
# import ace_tools as tools
with sqlite3.connect(path_db) as conn:
        df = pd.read_sql_query(
            """
            SELECT sentence_id,
                   sentiment_score,
                   sentiment_label,
                   sentence_text
            FROM   Sentences
            ORDER  BY sentence_id
            LIMIT  10;
            """, conn)
print(df)

   sentence_id  sentiment_score sentiment_label  \
0            1             -2.0        negative   
1            2             -2.8        negative   
2            3             -1.8        negative   
3            4              NaN            None   
4            5             -1.0        negative   
5            6              2.8        positive   
6            7              0.0         neutral   
7            8              0.0         neutral   
8            9             -1.8        negative   
9           10              NaN            None   

                                       sentence_text  
0  Con gái bầu Đức gom bất thành 4 triệu cp HAG v...  
1  Bà Đoàn Hoàng Anh, con gái ông Đoàn Nguyên Đức...  
2  Theo công bố thông tin, bà Đoàn Hoàng Anh- con...  
3  Lý do được đưa ra là không thu xếp được tài ch...  
4  Sau giao dịch bất thành, bà Hoàng Anh vẫn giữ ...  
5  Trước đó, từ 30/12/2024-21/01/2025, bà đã mua ...  
6  Hiện tại, ông Đoàn Nguyên Đức là cổ đông lớn n... 